In [ ]:
import random # to choose random responses from json file
import json  # to import json file
import pickle # for serealization
import numpy as np

In [ ]:
import nltk

In [ ]:
from nltk.stem import WordNetLemmatizer

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD


In [ ]:
#create object of lemmatizer
lemmatizer=WordNetLemmatizer()

In [ ]:
intents=json.loads(open('intents.json').read()) #reading the context from json file

In [ ]:
#intents

In [ ]:
words = []
classes = []
documents = []
ignore_letters = ['?','!','.',',']


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
for intent in intents['intents']:
  for pattern in intent['patterns']:
    word_list=nltk.word_tokenize(pattern)
    words.extend(word_list)
    documents.append((word_list, intent['tag']))
    if intent['tag'] not in classes:
      classes.append(intent['tag'])
print(documents)
print("-----------------------")
print(documents[0])

[(['hello'], 'greetings'), (['hey'], 'greetings'), (['hi'], 'greetings'), (['good', 'day'], 'greetings'), (['greetings'], 'greetings'), (['what', "'s", 'up', '?'], 'greetings'), (['hoe', 'is', 'it', 'going', '?'], 'greetings'), (['cya'], 'Goodbye!'), (['see', 'you', 'later'], 'Goodbye!'), (['goodbye'], 'Goodbye!'), (['I', 'am', 'leaving'], 'Goodbye!'), (['have', 'a', 'good', 'day'], 'Goodbye!'), (['bye'], 'Goodbye!'), (['cao'], 'Goodbye!'), (['see', 'ya'], 'Goodbye!'), (['how', 'old'], 'age'), (['how', 'old', 'is', 'florian'], 'age'), (['what', 'is', 'your', 'age'], 'age'), (['how', 'old', 'are', 'you'], 'age'), (['age', '?'], 'age'), (['what', 'is', 'your', 'name'], 'name'), (['what', 'should', 'i', 'call', 'you'], 'name'), (['what', 'is', 'your', 'name'], 'name'), (['who', 'are', 'you'], 'name'), (['can', 'you', 'tell', 'me', 'your', 'name', '?'], 'name'), (['I', 'do', 'like', 'to', 'buy', 'something'], 'shop'), (['what', 'are', 'your', 'products', '?'], 'shop'), (['what', 'do', 'you

In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
words=[lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words=sorted(set(words))
print(words)

["'s", 'I', 'a', 'age', 'am', 'are', 'buy', 'bye', 'call', 'can', 'cao', 'cya', 'day', 'do', 'florian', 'going', 'good', 'goodbye', 'greeting', 'guy', 'have', 'hello', 'hey', 'hi', 'hoe', 'hour', 'how', 'i', 'is', 'it', 'later', 'leaving', 'like', 'me', 'name', 'of', 'old', 'open', 'operation', 'product', 'recommend', 'see', 'selling', 'should', 'something', 'tell', 'to', 'up', 'what', 'when', 'who', 'ya', 'you', 'your']


In [ ]:
classes=sorted(set(classes))
print(classes)

['Goodbye!', 'age', 'greetings', 'hours', 'name', 'shop']


In [ ]:
pickle.dump(words, open('words.pkl','wb'))
pickle.dump(classes, open('classes.pkl','wb'))

In [ ]:
#bag of words
training=[]
output_empty=[0]*(len(classes))
#print(output_row)
for document in documents:
  bag=[]
  word_patterns=document[0]
  word_patterns=[lemmatizer.lemmatize(word.lower()) for word in word_patterns]
  for word in words:
    bag.append(1) if word in word_patterns else bag.append(0)
  
  output_row=list(output_empty)
  #print(output_row)
  output_row[classes.index(document[1])]=1

  training.append([bag, output_row])

[0, 0, 0, 1, 0, 0]


In [ ]:
#train the ml model
random.shuffle(training)
training=np.array(training)

train_x=list(training[:,0])
train_y=list(training[0:,1])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:

#from tensorflow.keras.optimizers import SGD
import tensorflow as tf
model=Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),),activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])



In [ ]:
model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 128)               7040      
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                                                 
 dense_8 (Dense)             (None, 64)                8256      
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_9 (Dense)             (None, 6)                 390       
                                                                 
Total params: 15,686
Trainable params: 15,686
Non-trainable params: 0
_________________________________________________________________


In [ ]:
hist=model.fit(np.array(train_x), np.array(train_y), epochs=300, batch_size=5,verbose=1)

model.save('chatbot_model.h5',hist)
print("Done!")

In [ ]:
#Chatbot application to use train model
import random
import numpy as np
import json
import pickle
import nltk
from nltk.stem import WordNetLemmatizer

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
lemmatizer=WordNetLemmatizer()
intents=json.loads(open('intents.json').read())

In [ ]:
words=pickle.load(open('words.pkl','rb'))
classes=pickle.load(open('classes.pkl','rb'))
model=load_model('chatbot_model.model')
#words

In [ ]:
def clean_up_sentence(sentence):
  sentence_words=nltk.word_tokenize(sentence)
  sentence_words=[lemmatizer.lemmatize(word) for word in sentence_words]
  return sentence_words

In [ ]:
def bag_of_words(sentence):
  sentence_words=clean_up_sentence(sentence)
  bag=[0]*len(words)
  for w in sentence_words:
    for i, w in enumerate(words):
      if w==words:
        bag[i]=1
  return np.array(bag)

In [ ]:
def predict_class(sentence):
  bow=bag_of_words(sentence)
  res=model.predict(np.array([bow]))[0]
  ERROR_THRESHOLD=0.25 
  results=[[i,r] for i, r in enumerate (res) if r > ERROR_THRESHOLD]

  results.sort(key=lambda x:x[1],reverse=True)
  return_list=[]
  for r in results:
    return_list.append({'intent':classes[r[0]],'probability':str(r[1])})
  return return_list
#print(return_list)
  #return res



In [ ]:
def get_response(intents_list,intents_json):
  tag=intents_list[0]['intent']
  list_of_intents=intents_json['intents']
  for i in list_of_intents:
    if i['tag']==tag:
      result=random.choice(i['responses'])
      break
  return result

In [ ]:
print("GO! BOT IS RUNNING!")

GO! BOT IS RUNNING!


In [ ]:
while True:
  message=input(" ")
  ints=predict_class(message)
  res=get_response(ints,intents)
  print(res)

 hello
Hello
 hey
hey
 hii
hey
 i need your help
what can i do for you?
 shop time
what can i do for you?


KeyboardInterrupt: ignored